In [2]:
import pandas as pd
data = pd.read_csv('preprocessed_data.csv')
data.head()

,temperature_2m,relative_humidity_2m,dew_point_2m,surface_pressure,cloud_cover,wind_speed_10m,hour,month,weather_group,temperature_2m_lag1,...,relative_humidity_2m_lag2,surface_pressure_lag1,surface_pressure_lag2,pressure_trend,humidity_change,hour_sin,hour_cos,dew_point_gap,pressure_trend_6h,humidity_roll_mean_3h
0,21.6,92,20.2,1014.4,100,4.3,6,1,Drizzle,22.9,...,90.0,1013.9,1013.4,0.8,5.0,1.000000,6.123234e-17,1.4,-0.6,89.666667
1,22.5,90,20.9,1014.9,100,2.9,7,1,Drizzle,21.6,...,87.0,1014.4,1013.9,1.5,-2.0,0.965926,-2.588190e-01,1.6,0.2,89.666667
2,23.5,89,21.5,1016.3,98,5.6,8,1,Drizzle,22.5,...,92.0,1014.9,1014.4,2.4,-1.0,0.866025,-5.000000e-01,2.0,2.3,90.333333
3,23.9,87,21.6,1016.5,83,6.4,9,1,Drizzle,23.5,...,90.0,1016.3,1014.9,2.1,-2.0,0.707107,-7.071068e-01,2.3,2.9,88.666667
4,24.8,82,21.6,1016.1,93,9.2,10,1,Drizzle,23.9,...,89.0,1016.5,1016.3,1.2,-5.0,0.500000,-8.660254e-01,3.2,2.7,86.000000


In [5]:
X = data.drop(columns=['weather_group'])
y = data['weather_group'].astype('category').cat.codes # Chuyển nhãn chữ sang số

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42, shuffle=False)


In [11]:
from sklearn.preprocessing import RobustScaler
numeric_features = ['temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'surface_pressure', 'cloud_cover', 'wind_speed_10m',
       'pressure_trend', 'humidity_change', 'temperature_2m_lag1',
       'temperature_2m_lag2', 'relative_humidity_2m_lag1',
       'relative_humidity_2m_lag2', 'surface_pressure_lag1',
       'surface_pressure_lag2', 'dew_point_gap',
       'pressure_trend_6h', 'humidity_roll_mean_3h']

robust_scaler = RobustScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numeric_features] = robust_scaler.fit_transform(X_train[numeric_features])
X_val_scaled[numeric_features] = robust_scaler.transform(X_val[numeric_features])
X_test_scaled[numeric_features] = robust_scaler.transform(X_test[numeric_features])


## 4. Softmax Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time
import numpy as np

### 4.1 Tìm kiếm siêu tham số (Hyperparameter Tuning)

Để tìm ra bộ tham số tối ưu cho mô hình Softmax Regression, chúng tôi sử dụng kỹ thuật **Grid Search** kết hợp với tập validation đã được chia từ trước.

**Phương pháp:**
- **Grid Search**: Thử nghiệm exhaustive trên một lưới các giá trị tham số được định nghĩa trước.
- **PredefinedSplit**: Vì dữ liệu là chuỗi thời gian (time-series), việc xáo trộn (shuffle) ngẫu nhiên trong Cross-Validation thông thường có thể làm rò rỉ dữ liệu hoặc phá vỡ tính chất thời gian. Do đó, chúng tôi sử dụng `PredefinedSplit` để chỉ định rõ ràng tập Validation (X_val) dùng để đánh giá từng bộ tham số, trong khi tập Train (X_train) dùng để huấn luyện.

**Các siêu tham số được tinh chỉnh:**
1.  **C** (Regularization Strength): Kiểm soát mức độ "phạt" của mô hình đối với các trọng số lớn. Giá trị `C` nhỏ tương ứng với regularization mạnh (giảm overfitting), `C` lớn tương ứng với regularization yếu.
    - Grid: `[0.01, 0.1, 1, 10, 100]`
2.  **solver**: Thuật toán tối ưu hóa dùng để tìm trọng số.
    - Grid: `['lbfgs', 'newton-cg']` (Các thuật toán này hỗ trợ tốt cho bài toán đa lớp multinomial).
3.  **max_iter**: Số vòng lặp tối đa để thuật toán hội tụ.
    - Value: `1000`


In [32]:
# Tạo các biến tương tác quan trọng
X_train_val['temp_humidity_interaction'] = X_train_val['temperature_2m'] * X_train_val['relative_humidity_2m']
X_train_val['pressure_humidity_interaction'] = X_train_val['surface_pressure'] * X_train_val['relative_humidity_2m']
# Nhớ áp dụng tương tự cho X_test_scaled

In [ ]:
# Chuẩn bị dữ liệu cho PredefinedSplit
# Ghép nối X_train và X_val để đưa vào GridSearchCV
X_train_val = pd.concat([X_train_scaled, X_val_scaled])
y_train_val = pd.concat([y_train, y_val])

# Tạo mảng 'test_fold' để chỉ định tập validation
# -1: mẫu thuộc tập train (không dùng để validate)
# 0: mẫu thuộc tập validation
split_index = [-1] * len(X_train_scaled) + [0] * len(X_val_scaled)
tscv = TimeSeriesSplit(n_splits=5)

# Thử nghiệm các tổ hợp trọng số khác nhau
# Ví dụ: Tăng nhẹ lớp 1 và 2 nhưng không quá gắt như 'balanced'
custom_weights = [{0: 1, 1: 1.2, 2: 1.5}, {0: 1, 1: 1.5, 2: 2}]

param_grid = {
    'C': [0.001, 0.01, 0.1, 1],
    'class_weight': [None, 'balanced'] + custom_weights, # Thử cả trọng số tùy chỉnh
    'solver': ['lbfgs', 'newton-cg']
}

softmax_reg = LogisticRegression(
    solver='lbfgs', 
    class_weight=custom_weights, 
    max_iter=1000
)
grid_search = GridSearchCV(
    estimator=softmax_reg,
    param_grid=param_grid,
    cv=tscv, # Sử dụng tscv ở đây
    scoring='accuracy', # Khuyên dùng f1_macro cho dữ liệu mất cân bằng
    n_jobs=-1
)

# Thực hiện tìm kiếm
start_search = time.time()
grid_search.fit(X_train_val, y_train_val)
end_search = time.time()

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score: {grid_search.best_score_:.4f}")
print(f"Search time: {end_search - start_search:.2f}s")

best_model_softmax = grid_search.best_estimator_

Best parameters: {'C': 0.01, 'max_iter': 1000, 'solver': 'newton-cg'}
Best validation score: 0.7396
Search time: 25.24s


In [34]:
from sklearn.metrics import f1_score, make_scorer

# 1. Sửa danh sách trọng số tùy chỉnh để bao gồm cả trường hợp không dùng trọng số
custom_weights = [{0: 1, 1: 1.2, 2: 1.5}, {0: 1, 1: 1.5, 2: 2.5}]

# 2. Định nghĩa lưới tham số tập trung vào giảm Overfitting
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10], # Thêm các giá trị nhỏ để giảm Overfitting
    'class_weight': [None, 'balanced'] + custom_weights, 
    'solver': ['lbfgs', 'newton-cg'],
    'max_iter': [2000] # Tăng số vòng lặp để đảm bảo hội tụ với C nhỏ
}

softmax_reg = LogisticRegression(solver='lbfgs', max_iter=2000)

# 4. Thiết lập GridSearchCV với f1_macro
# Lưu ý: Nên đổi scoring sang 'f1_macro' để so sánh công bằng với AdaBoost/RF
grid_search = GridSearchCV(
    estimator=softmax_reg,
    param_grid=param_grid,
    cv=tscv, 
    scoring='f1_macro', 
    n_jobs=-1,
    verbose=1
)

# 5. Thực hiện tìm kiếm
start_search = time.time()
grid_search.fit(X_train_val, y_train_val)
end_search = time.time()

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score (F1-Macro): {grid_search.best_score_:.4f}")

best_model_softmax = grid_search.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'C': 10, 'class_weight': None, 'max_iter': 2000, 'solver': 'lbfgs'}
Best validation score (F1-Macro): 0.4582


In [35]:
# Xem tầm quan trọng của các biến trong Softmax
for i, class_label in enumerate(best_model_softmax.classes_):
    print(f"\nTop features cho lớp {class_label}:")
    top_indices = np.argsort(np.abs(best_model_softmax.coef_[i]))[-5:]
    for idx in top_indices[::-1]:
        print(f"{X.columns[idx]}: {best_model_softmax.coef_[i][idx]:.4f}")


Top features cho lớp 0:
dew_point_gap: 3.1946
temperature_2m_lag1: -3.1918
temperature_2m: 2.9607
surface_pressure: 1.9047
surface_pressure_lag1: -1.6706

Top features cho lớp 1:
temperature_2m_lag1: -1.6246
temperature_2m_lag2: 0.8369
surface_pressure_lag1: 0.7557
relative_humidity_2m: -0.5821
surface_pressure: 0.5655

Top features cho lớp 2:
temperature_2m_lag1: 4.8164
temperature_2m: -3.3750
dew_point_gap: -3.2626
surface_pressure: -2.4701
dew_point_2m: -1.7785


### 4.2 Kiểm thử mô hình (Model Evaluation)

Sau khi chọn được mô hình tốt nhất, chúng tôi tiến hành đánh giá trên tập kiểm thử (Test Set) hoàn toàn độc lập.

**Các chỉ số đánh giá (Metrics):**
1.  **Accuracy (Độ chính xác)**: Tỷ lệ số mẫu dự đoán đúng trên tổng số mẫu. Đo lường hiệu suất tổng quan.
2.  **Training Time & Inference Time**: 
    - *Training Time*: Thời gian huấn luyện lại mô hình tốt nhất.
    - *Inference Time*: Thời gian mô hình dự đoán trên tập test.
3.  **Precision (Độ chính xác của từng lớp)**: Trong số các mẫu được dự đoán là lớp A, bao nhiêu % thực sự là lớp A. Quan trọng khi muốn giảm thiểu báo động giả (False Positives).
4.  **Recall (Độ phủ)**: Trong số các mẫu thực sự là lớp A, bao nhiêu % được mô hình phát hiện đúng. Quan trọng khi muốn tránh bỏ sót (False Negatives).
5.  **F1-Score**: Trung bình điều hòa của Precision và Recall. Cân bằng giữa độ chính xác và độ phủ, hữu ích khi dữ liệu mất cân bằng.
6.  **Confusion Matrix**: Bảng ma trận hiển thị chi tiết sự nhầm lẫn giữa các lớp.

In [30]:
# Huấn luyện lại mô hình tốt nhất (hoặc sử dụng kết quả từ GridSearch đã fit trên Train+Val)
# Ở đây GridSearch refit=True mặc định sẽ fit lại trên toàn bộ dữ liệu đưa vào (Train + Val)
# Đo thời gian Inference
start_inf = time.time()
y_pred = best_model_softmax.predict(X_test_scaled)
end_inf = time.time()
inference_time = end_inf - start_inf

# Đánh giá trên Train và Test
train_acc = best_model_softmax.score(X_train_val, y_train_val)
test_acc = accuracy_score(y_test, y_pred)

print(f"Training Accuracy (Result of GridSearch refit): {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")
print(f"Inference Time (on test set): {inference_time:.4f}s")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

Training Accuracy (Result of GridSearch refit): 0.6646
Testing Accuracy: 0.5093
Inference Time (on test set): 0.0079s

Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.34      0.48      1675
           1       0.48      0.59      0.53      1366
           2       0.36      0.87      0.51       472

    accuracy                           0.51      3513
   macro avg       0.56      0.60      0.51      3513
weighted avg       0.64      0.51      0.50      3513


Confusion Matrix:

[[566 824 285]
 [ 98 811 457]
 [  7  53 412]]
